In [ ]:
import flapjack
from flapjack import Flapjack
import pandas as pd
import numpy as np
import plotly.express as px

## Connecting to Flapjack

In [ ]:
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username='tim', password='chicken')

## Query Flapjack by filtering and searching

In [ ]:
od = fj.query('signal', name='OD')
cfp = fj.query('signal', search='CFP')
study = fj.query('study')
chems = fj.query('chemical', search='iptg')
media = fj.query('media', search='glucosa')
assay = fj.query('assay')
chems

In [ ]:
chemical = fj.query('chemical', pubchemid=12345)
chemical

## Plot the data

In [ ]:
fig = fj.plot(assay=assay.id,
                    media=media.id,
                    normalize='None', 
                    subplots='Signal', 
                    markers='Vector', 
                    plot='Mean')

In [ ]:
fig.show()

## Analyse the data and plot the result

In [ ]:
fig = fj.plot(assay=assay.id,
                    media=media.id, 
                    type='Mean Expression',
                    biomass_signal=od.id,
                    ref_signal=cfp.id,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False,
                    normalize='None', 
                    subplots='Signal', 
                    markers='Vector', 
                    plot='Mean')

In [ ]:
fig.show()

## Analyze the data and get a dataframe of the result

In [ ]:
df = fj.analysis(assay=[assay.id[0]],
                    media=media.id, 
                    type='Alpha',
                    biomass_signal=od.id,
                    ref_signal=cfp.id,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False
                      )


### Custom plot the results

In [ ]:
px.bar(df, x='Vector', y='Alpha', facet_col='Signal')

### Statistical analysis of the results (TO DO)

## Compute an induction curve and fit a model

In [ ]:
dna = fj.query('dna', name='CcaT')
vector = fj.query('vector', dnas=dna.id)
chemical = fj.query('chemical', name='IPTG')
biomass_signal = fj.query('signal', name='OD')

df = fj.analysis(vector=vector.id,
                 analyte=chemical.id,
                 type='Induction Curve',
                 function='Mean Expression',
                 biomass_signal=biomass_signal.id
                )

### Fit a model to the data
The fit_curve function is a convenient wrapper for SciPy's curve_fit function. Here you may pass optional keyword arguments (such as p0, bounds) to curve_fit (see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html). The function returns the fitted parameters and the estimated standard deviation.

Flapjack provides several common functions such as the Hill function used here, but you may specify any function just as with curve_fit. The parameters x and y are the columns that are used for the x and y values to fit the function.

In [ ]:
import matplotlib.pyplot as plt
data = df[df.Signal=='Turquoise'][df.Vector=='CcaT+TMA5']
params,std = flapjack.fit_curve(flapjack.hill, data, x='Concentration', y='Expression')

In [ ]:
print(params)
print(std)

### Plot the result of the fit

In [ ]:
data.plot(x='Concentration', y='Expression', label='CcaT+TMA5', style='+', logx=True)
x = np.logspace(-6,-3,100)
y = flapjack.hill(x, *params)
plt.plot(x, y, 'k--')

## Connecting to SynbioHub

In [ ]:
from sbol import *

dna = fj.query('dna', name='pAAA')
sboluri = dna.sboluri

synbiohub = PartShop('https://synbiohub.org')
doc = Document()
synbiohub.pull(sboluri, doc)

## Creating new studies and associated assays

### Create a new study

In [ ]:
study = fj.create('study', name='Another test study', description='Testing the API again')
fj.query('study')

### Add an assay to the study

In [ ]:
assay = fj.create('assay',
                  name='Test',
                  description='A test',
                  machine='Test',
                  temperature=12,
                  study=study.id[0]
                 )
fj.query('assay')

### Delete the study

In [ ]:
fj.delete('study', study.id[0])
fj.query('study')

This also deletes all the assays, samples and measurements in the study.

In [ ]:
fj.query('assay')